In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [3]:
# Load your dataset
with open('/content/sample_data/1661-0.txt', 'r') as file:
    text = file.read().lower()

In [4]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1


In [5]:
# Create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
# Pad sequences and create predictors and labels
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [7]:
# Build the RNN model
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Train the model
history = model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
3176/3176 [==============================] - 80s 24ms/step - loss: 6.3563 - accuracy: 0.0681
Epoch 2/10
3176/3176 [==============================] - 77s 24ms/step - loss: 5.6571 - accuracy: 0.1101
Epoch 3/10
3176/3176 [==============================] - 76s 24ms/step - loss: 5.3300 - accuracy: 0.1306
Epoch 4/10
3176/3176 [==============================] - 79s 25ms/step - loss: 5.0771 - accuracy: 0.1451
Epoch 5/10
3176/3176 [==============================] - 96s 30ms/step - loss: 4.8592 - accuracy: 0.1567
Epoch 6/10
3176/3176 [==============================] - 82s 26ms/step - loss: 4.6690 - accuracy: 0.1668
Epoch 7/10
3176/3176 [==============================] - 77s 24ms/step - loss: 4.4977 - accuracy: 0.1768
Epoch 8/10
3176/3176 [==============================] - 78s 24ms/step - loss: 4.3397 - accuracy: 0.1875
Epoch 9/10
3176/3176 [==============================] - 78s 25ms/step - loss: 4.1960 - accuracy: 0.1998
Epoch 10/10
3176/3176 [==============================] - 89s 28m

In [9]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return ""

In [10]:
# Test the model
seed_text = "This eBook is for the use of"
next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_len)
print(seed_text + " " + next_word)

This eBook is for the use of anyone
